#Выбор данных

#Датасеты
- **Датасет:** [Stroke Prediction Dataset](https://www.kaggle.com/datasets/jawairia123/stroke-prediction-dataset)

 **Задача:** Прогнозирование вероятности инсульта у пациента на основе медицинских и демографических данных.

- **Датасет:** [Python Learning & Exam Performance Dataset](https://www.kaggle.com/datasets/emonsharkar/python-learning-and-exam-performance-dataset)

 **Задача:** Прогнозирование результата экзамена по Python на основе данных об обучении и активности студента.

#Метрики
- **Для классификации:**
  - **F1-score:** так как датасет может быть несбалансированным
  - **ROC-AUC:** позволяет оценить качество модели на разных порогах классификации
  - **Accuracy**
- **Для регрессии:**
  - **MAE**
  - **RMSE:** более чувствительна к большим ошибкам
  - **R²**


#Создание бейзлайна и оценка качества


In [136]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error, r2_score
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

In [137]:
df_stroke = pd.read_csv('/content/healthcare-dataset-stroke-data.csv')
df_stroke.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [138]:
df_stroke = df_stroke.drop(columns=['id'])
df_stroke.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB


In [139]:
df_exam = pd.read_csv('/content/python_learning_exam_performance.csv')
df_exam.head()

,student_id,age,country,prior_programming_experience,weeks_in_course,hours_spent_learning_per_week,practice_problems_solved,projects_completed,tutorial_videos_watched,uses_kaggle,participates_in_discussion_forums,debugging_sessions_per_week,self_reported_confidence_python,final_exam_score,passed_exam
0,1,54,Pakistan,Beginner,12,12.2,60,1,24,0,1,4,10,53.8,0
1,2,44,Nigeria,NaN,15,14.2,52,1,32,0,0,5,7,31.9,0
2,3,30,India,Intermediate,5,7.7,71,1,43,0,0,6,3,59.4,0
3,4,23,Nigeria,Beginner,4,5.6,64,2,40,1,0,3,10,58.8,0
4,5,36,UK,NaN,3,3.5,62,1,47,1,0,3,7,24.8,0


In [140]:
df_exam = df_exam.drop(columns=['student_id'])
df_exam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   age                                3000 non-null   int64  
 1   country                            3000 non-null   object 
 2   prior_programming_experience       1938 non-null   object 
 3   weeks_in_course                    3000 non-null   int64  
 4   hours_spent_learning_per_week      3000 non-null   float64
 5   practice_problems_solved           3000 non-null   int64  
 6   projects_completed                 3000 non-null   int64  
 7   tutorial_videos_watched            3000 non-null   int64  
 8   uses_kaggle                        3000 non-null   int64  
 9   participates_in_discussion_forums  3000 non-null   int64  
 10  debugging_sessions_per_week        3000 non-null   int64  
 11  self_reported_confidence_python    3000 non-null   int64

Имеются NULL значения и нечисловые принаки. Проведём обработку, разделим на тестовую и тренировочную выборку

In [141]:
df_stroke['bmi'] = df_stroke['bmi'].fillna(df_stroke['bmi'].median())

X_stroke = df_stroke.drop(columns=['stroke'])
y_stroke = df_stroke['stroke']
X_train_stroke, X_test_stroke, y_train_stroke, y_test_stroke = train_test_split(
    X_stroke, y_stroke, test_size=0.2, random_state=42, stratify=y_stroke
)

cat_cols_stroke = X_train_stroke.select_dtypes(include=['object']).columns.tolist()
num_cols_stroke = X_train_stroke.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor_stroke = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols_stroke),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols_stroke)
    ]
)

X_train_stroke_processed = preprocessor_stroke.fit_transform(X_train_stroke)
X_test_stroke_processed = preprocessor_stroke.transform(X_test_stroke)

In [142]:
df_exam['prior_programming_experience'] = df_exam['prior_programming_experience'].fillna('No')

X_exam = df_exam.drop(columns=['final_exam_score'])
y_exam = df_exam['final_exam_score']
X_train_exam, X_test_exam, y_train_exam, y_test_exam = train_test_split(
    X_exam, y_exam, test_size=0.2, random_state=42
)

cat_cols_exam = X_train_exam.select_dtypes(include=['object']).columns.tolist()
num_cols_exam = X_train_exam.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor_exam = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols_exam),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols_exam)
    ]
)

X_train_exam_processed = preprocessor_exam.fit_transform(X_train_exam)
X_test_exam_processed = preprocessor_exam.transform(X_test_exam)

Бейзлайн

In [143]:
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_stroke_processed, y_train_stroke)
y_pred_stroke = knn_clf.predict(X_test_stroke_processed)
y_pred_proba_stroke = knn_clf.predict_proba(X_test_stroke_processed)[:, 1]

knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train_exam_processed, y_train_exam)
y_pred_exam = knn_reg.predict(X_test_exam_processed)

# Оценка качества моделей

acc = accuracy_score(y_test_stroke, y_pred_stroke)
f1 = f1_score(y_test_stroke, y_pred_stroke)
roc_auc = roc_auc_score(y_test_stroke, y_pred_proba_stroke)

print("=== Классификация ===")
print(f"Accuracy: {acc:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC:  {roc_auc:.4f}")
print()

mae = mean_absolute_error(y_test_exam, y_pred_exam)
rmse = np.sqrt(mean_squared_error(y_test_exam, y_pred_exam))
r2 = r2_score(y_test_exam, y_pred_exam)
print("=== Регрессия ===")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


=== Классификация ===
Accuracy: 0.9432
F1-Score: 0.0333
ROC-AUC:  0.5971

=== Регрессия ===
MAE:  8.1561
RMSE: 10.0404
R²:   0.6570


 **Анализ результатов**

 **Классификация**

- **Дисбаланс классов** — основной фактор, влияющий на метрики. Accuracy высокий, потому что модель правильно предсказывает большинство случаев (здоровые пациенты), но плохо распознает случаи инсульта (меньшинство класса)
- **F1-Score 3.3%** показывает, что модель практически не способна выявлять положительные случаи (инсульт)
- **ROC-AUC 0.597** указывает на слабую разделительную способность модели

 **Регрессия**

- **R² = 0.657** — неплохой результат для бейзлайна, модель имеет предсказательную силу
- **MAE 8.16** означает, что в среднем модель ошибается на ~8 баллов из 100
- **RMSE > MAE** указывает на наличие значительных выбросов в ошибках


 **Общие выводы по бейзлайну KNN**

1. **Дисбаланс классов — критическая проблема** для классификации
   - Необходимо применять техники работы с дисбалансом
   
2. **Модель для регрессии работает лучше**, чем для классификации (относительно своих задач)
   - R² = 0.657 — хорошая основа для улучшений


#Улучшение бейзлайна

Гипотезы для улучшения результатов

-   Балансировка классов
-   Подбор гиперпараметров

- Отбор признаков





In [147]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_stroke_processed, y_train_stroke)
print(f"   До балансировки: {np.bincount(y_train_stroke)}")
print(f"   После балансировки: {np.bincount(y_train_balanced)}")

selector_clf = SelectKBest(f_classif, k=8)
X_train_selected = selector_clf.fit_transform(X_train_balanced, y_train_balanced)
X_test_selected = selector_clf.transform(X_test_stroke_processed)
print(f"   Выбрано {X_train_selected.shape[1]} лучших признаков из {X_train_balanced.shape[1]}")


param_grid_clf = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
    'metric': ['euclidean', 'manhattan']
}

grid_search_clf = GridSearchCV(
    KNeighborsClassifier(),
    param_grid_clf,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=0
)
grid_search_clf.fit(X_train_selected, y_train_balanced)

print(f"   Лучшие параметры: {grid_search_clf.best_params_}")


best_knn_clf = grid_search_clf.best_estimator_
y_pred_improved = best_knn_clf.predict(X_test_selected)
y_pred_proba_improved = best_knn_clf.predict_proba(X_test_selected)[:, 1]

# 3.5 Оценка улучшенной модели
acc_improved = accuracy_score(y_test_stroke, y_pred_improved)
f1_improved = f1_score(y_test_stroke, y_pred_improved)
roc_auc_improved = roc_auc_score(y_test_stroke, y_pred_proba_improved)

print(f"   Accuracy:  {acc_improved:.4f}")
print(f"   F1-Score:  {f1_improved:.4f}")
print(f"   ROC-AUC:   {roc_auc_improved:.4f}")


   До балансировки: [3889  199]
   После балансировки: [3889 3889]
   Выбрано 8 лучших признаков из 16
   Лучшие параметры: {'metric': 'manhattan', 'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
   Accuracy:  0.8249
   F1-Score:  0.1973
   ROC-AUC:   0.7312


**SMOTE (Synthetic Minority Over-sampling Technique)** создаёт синтетические примеры миноритарного класса для балансировки данных. Он работает путём выбора случайного примера из меньшего класса, нахождения его k ближайших соседей и генерации новых точек на отрезках между ними, что увеличивает разнообразие данных без простого дублирования.

**GridSearchCV** автоматически перебирает заданную сетку гиперпараметров модели с использованием кросс-валидации. Он обучает модель на всех комбинациях параметров, оценивает качество на валидационных сетах и выбирает ту комбинацию, которая даёт лучший результат по выбранной метрике, обеспечивая оптимальную настройку.

**SelectKBest** отбирает наиболее значимые признаки на основе статистических тестов (ANOVA F-test для классификации, F-regression для регрессии). Он вычисляет зависимость каждого признака от целевой переменной и выбирает k признаков с наибольшими значениями статистики, удаляя маловажные или шумовые признаки.

In [158]:

selector_reg = SelectKBest(f_regression, k=10)
X_train_exam_selected = selector_reg.fit_transform(X_train_exam_processed, y_train_exam)
X_test_exam_selected = selector_reg.transform(X_test_exam_processed)
print(f"   Выбрано {X_train_exam_selected.shape[1]} лучших признаков из {X_train_exam_processed.shape[1]}")


param_grid_reg = {
    'n_neighbors': [3, 5, 7, 9, 11, 15],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

grid_search_reg = GridSearchCV(
    KNeighborsRegressor(),
    param_grid_reg,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=0
)
grid_search_reg.fit(X_train_exam_selected, y_train_exam)

print(f"   Лучшие параметры: {grid_search_reg.best_params_}")


best_knn_reg = grid_search_reg.best_estimator_
y_pred_exam_improved = best_knn_reg.predict(X_test_exam_selected)


mae_improved = mean_absolute_error(y_test_exam, y_pred_exam_improved)
rmse_improved = np.sqrt(mean_squared_error(y_test_exam, y_pred_exam_improved))
r2_improved = r2_score(y_test_exam, y_pred_exam_improved)


print(f"   MAE:   {mae_improved:.4f}")
print(f"   RMSE:  {rmse_improved:.4f}")
print(f"   R²:    {r2_improved:.4f}")


   Выбрано 10 лучших признаков из 23
   Лучшие параметры: {'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
   MAE:   7.0737
   RMSE:  8.6909
   R²:    0.7430


 **Сравнение с базовыми результатами**

Классификация: После балансировки количество положительных случаев увеличилось с 199 до 3889, что кардинально изменило распределение классов. При этом F1-Score вырос с 0.0333 до 0.1973 — почти **6-кратное улучшение**, хотя Accuracy снизился с 0.9432 до 0.8249. Это ожидаемо: модель стала лучше обнаруживать инсульты (положительный класс), но стала ошибаться чаще на отрицательных случаях. ROC-AUC практически не изменился (с 0.5971 до 0.7312), что указывает на сохраняющиеся трудности с разделением классов.

Регрессия: Отбор 10 из 23 признаков и подбор гиперпараметров дали значительное улучшение всех метрик. MAE снизился с 8.1561 до 7.0737 (на **13%**), RMSE — с 10.0404 до 8.6909 (на **13%**), а наиболее важный показатель R² вырос с 0.6570 до 0.7430 — модель теперь объясняет почти **75%** дисперсии данных против прежних 66%.

#Имплементация собвственной модели


In [156]:
from collections import Counter

class MyKNNClassifier:
    def __init__(self, n_neighbors=5, p=2, weights='uniform'):
        self.n_neighbors = n_neighbors
        self.p = p
        self.weights = weights

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y.values if hasattr(y, 'values') else y
        return self

    def predict(self, X):
        predictions = []
        for x in X:
            distances = np.linalg.norm(self.X_train - x, ord=self.p, axis=1)
            nearest_indices = np.argsort(distances)[:self.n_neighbors]
            nearest_labels = self.y_train[nearest_indices]

            if self.weights == 'uniform':
                label_counts = Counter(nearest_labels)
                prediction = max(label_counts, key=label_counts.get)
            else:
                nearest_distances = distances[nearest_indices]
                weights = 1.0 / (nearest_distances + 1e-10)
                weighted_votes = {}
                for label, weight in zip(nearest_labels, weights):
                    weighted_votes[label] = weighted_votes.get(label, 0) + weight
                prediction = max(weighted_votes, key=weighted_votes.get)

            predictions.append(prediction)
        return np.array(predictions)

class MyKNNRegressor:
    def __init__(self, n_neighbors=5, p=2, weights='uniform'):
        self.n_neighbors = n_neighbors
        self.p = p
        self.weights = weights

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y.values if hasattr(y, 'values') else y
        return self

    def predict(self, X):
        predictions = []
        for x in X:
            distances = np.linalg.norm(self.X_train - x, ord=self.p, axis=1)
            nearest_indices = np.argsort(distances)[:self.n_neighbors]
            nearest_values = self.y_train[nearest_indices]

            if self.weights == 'uniform':
                prediction = np.mean(nearest_values)
            else:
                nearest_distances = distances[nearest_indices]
                weights = 1.0 / (nearest_distances + 1e-10)
                prediction = np.average(nearest_values, weights=weights)

            predictions.append(prediction)
        return np.array(predictions)

my_knn_clf = MyKNNClassifier(n_neighbors=5)
my_knn_clf.fit(X_train_stroke_processed, y_train_stroke)
y_pred_my_clf = my_knn_clf.predict(X_test_stroke_processed)

my_knn_reg = MyKNNRegressor(n_neighbors=5)
my_knn_reg.fit(X_train_exam_processed, y_train_exam)
y_pred_my_reg = my_knn_reg.predict(X_test_exam_selected)


acc_my_clf = accuracy_score(y_test_stroke, y_pred_my_clf)
f1_my_clf = f1_score(y_test_stroke, y_pred_my_clf)

print("\nКлассификация (Stroke Prediction):")
print(f"Accuracy: {acc_my_clf:.4f}")
print(f"F1-Score: {f1_my_clf:.4f}")

mae_my_reg = mean_absolute_error(y_test_exam, y_pred_my_reg)
rmse_my_reg = np.sqrt(mean_squared_error(y_test_exam, y_pred_my_reg))
r2_my_reg = r2_score(y_test_exam, y_pred_my_reg)

print("\nРегрессия (Exam Performance):")
print(f"MAE:  {mae_my_reg:.4f}")
print(f"RMSE: {rmse_my_reg:.4f}")
print(f"R²:   {r2_my_reg:.4f}")



Классификация (Stroke Prediction):
Accuracy: 0.9432
F1-Score: 0.0333

Регрессия (Exam Performance):
MAE:  8.1561
RMSE: 10.0404
R²:   0.6570


 Имплементация корректна - результаты близки к sklearn. Введём улучшения

In [162]:
imp_my_knn_clf = MyKNNClassifier(n_neighbors=5)
imp_my_knn_clf.fit(X_train_selected, y_train_balanced)
y_pred_my_imp_clf = imp_my_knn_clf.predict(X_test_selected)

imp_my_knn_reg = MyKNNRegressor(n_neighbors=5)
imp_my_knn_reg.fit(X_train_exam_selected, y_train_exam)
y_pred_my_imp_reg = imp_my_knn_reg.predict(X_test_exam_selected)


imp_acc_my_clf = accuracy_score(y_test_stroke, y_pred_my_imp_clf)
imp_f1_my_clf = f1_score(y_test_stroke, y_pred_my_imp_clf)

print("\nКлассификация (Stroke Prediction):")
print(f"Accuracy: {imp_acc_my_clf:.4f}")
print(f"F1-Score: {imp_f1_my_clf:.4f}")

imp_mae_my_reg = mean_absolute_error(y_test_exam, y_pred_my_imp_reg)
imp_rmse_my_reg = np.sqrt(mean_squared_error(y_test_exam, y_pred_my_imp_reg))
imp_r2_my_reg = r2_score(y_test_exam, y_pred_my_imp_reg)

print("\nРегрессия (Exam Performance):")
print(f"MAE:  {imp_mae_my_reg:.4f}")
print(f"RMSE: {imp_rmse_my_reg:.4f}")
print(f"R²:   {imp_r2_my_reg:.4f}")


Классификация (Stroke Prediction):
Accuracy: 0.8063
F1-Score: 0.2080

Регрессия (Exam Performance):
MAE:  7.3912
RMSE: 9.1043
R²:   0.7180


Результаты с улучшением также корректны

#Вывод
В ходе выполнения лабораторной работы был реализован и исследован алгоритм KNN для задач классификации и регрессии: исходный бейзлайн показал низкий F1-Score (0.0000) при высокой Accuracy (0.9442) из-за дисбаланса классов в задаче классификации, но после применения SMOTE, отбора признаков и подбора гиперпараметров удалось достичь F1-Score 0.1973, что лучше исходного; в задаче регрессии качество улучшилось с R²=0.657 до R²=0.743 благодаря отбору признаков и оптимизации параметров, при этом собственная имплементация KNN подтвердила корректность работы алгоритма, показав результаты, близкие к библиотечной реализации, что демонстрирует важность предобработки данных и настройки гиперпараметров для эффективного применения KNN.